In [4]:
import pandas as pd
import numpy as np
from typing import List, Tuple

In [5]:
#### Parameters
spot_filename:str = "/tmp/index.csv"
options_filename:str = "../../data/nifty_options_eod.h5"
output_filename:str = '/tmp/output.csv' # Friendly name for output
step:int = 100
options:Tuple[Tuple[str, float]] = (
    ('p', 0),
    ('p', 0.01),
    ('p', 0.02),
    ('p', 0.03),
    ('c', 0.0),
    ('c', 0.01),
    ('c', 0.02),
    ('c', 0.03)
)


In [6]:
index = pd.read_csv(spot_filename, parse_dates=['date'])
opt = pd.read_hdf(options_filename).sort_values(by=['date'])
rename = {
    'open_price': 'open',
    'high_price': 'high',
    'low_price': 'low',
    'close_pric': 'close'
}
opt = opt.rename(columns=rename)
opt = opt[(opt.date.dt.year == opt.expiry_date.dt.year) &\
    (opt.date.dt.month == opt.expiry_date.dt.month)].sort_values(by='date')
opt['de'] = (opt['expiry_date'] - opt['date']).dt.days
opt['year'] = opt.date.dt.year
opt['month'] = opt.date.dt.month
opt['strike_pr'] = opt.contract_d.str[24:].astype(int)

In [7]:
expiry_dates = opt.groupby(['year', 'month']).expiry_date.max().reset_index()
opt2 = opt[opt.expiry_date.isin(expiry_dates.expiry_date.values)]

In [8]:
ce = opt2.query("opt_type=='CE'")
pe = opt2.query("opt_type=='PE'")

In [9]:
def get_result(options_data, spot_data, opt='p', itm=0, step=100):
    if opt.lower() == 'p':
        m = 1-itm
    else:
        m = 1+itm
        
    first_dates = options_data.groupby(['year', 'month']).date.min().reset_index()
    opts = options_data.merge(first_dates, on=['year', 'month', 'date'])
    opts['spot'] = opts.undrlng_st.copy()
    opts['strike'] = [int((x*m)/step)*step for x in opts.undrlng_st.values]
    if opt.lower() == 'p':
        opts['strike'] = opts['strike'] + step
    opts = opts.query('strike==strike_pr')
    cols = ['date', 'contract_d', 'strike', 'expiry_date']
    entries = opts[cols].copy().drop_duplicates()
    entries['entry_date'] = entries.date.copy()
    
    last_dates = options_data.groupby(['year', 'month']).date.max().reset_index()
    exits = entries.copy()
    del exits['entry_date']
    exits['date'] = last_dates.date.values
    exits['exit_date'] = exits.date.copy()

    frame1 = entries.merge(options_data, on=cols)
    frame2 = exits.merge(options_data, on=cols)
    trades = frame1.merge(frame2, on=['contract_d', 'expiry_date'])
    rename = { 
        'date_x': 'date',
        'close_x': 'entry_price',
        'undrlng_st_x': 'entry_spot',
        'close_y': 'exit_price',
        'undrlng_st_y': 'exit_spot'
    }
    trades = trades.rename(columns=rename)
    cols = ['date', 'contract_d', 'expiry_date', 'entry_date', 'entry_price',
           'exit_date', 'exit_price', 'entry_spot', 'exit_spot']
    trades['pnl'] = trades.eval('exit_price-entry_price')
    trades['de'] = (trades['expiry_date'] - trades['entry_date']).dt.days
    return trades[cols + ['pnl', 'de']]

In [10]:
collect = []
for i,(opt,strike) in enumerate(options):
    data = ce if opt =='c' else pe
    temp = get_result(data, index, itm=strike,step=step, opt=opt)
    temp['name'] = f"opt{i}"
    collect.append(temp)
res = pd.concat(collect)
del collect
res.pnl.describe()

count     528.000000
mean       11.617803
std       302.204925
min      -571.500000
25%      -131.312500
50%       -67.875000
75%        54.100000
max      2271.300000
Name: pnl, dtype: float64

In [11]:
res.groupby('name').pnl.describe()

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
opt0,66.0,-26.807576,376.276608,-571.50,-172.3875,-126.10,25.5250,2271.30
opt1,66.0,-28.750000,347.823329,-529.05,-134.8000,-107.30,-32.9375,2213.45
opt2,66.0,-26.088636,320.109340,-488.70,-115.6000,-84.45,-57.7125,2138.75
opt3,66.0,-20.638636,289.764402,-488.70,-94.2125,-58.75,-40.9500,1995.30
opt4,66.0,50.658333,318.721654,-423.90,-181.8125,-62.70,264.2250,1083.70
opt5,66.0,53.161364,279.171050,-319.60,-120.8500,-65.05,225.1125,1037.15
opt6,66.0,49.064394,242.036904,-271.50,-84.2750,-56.45,182.9125,991.45
opt7,66.0,42.343182,204.990340,-259.85,-58.5625,-35.50,92.8625,935.85


In [12]:
res.to_csv(output_filename)